In [ ]:
import pandas as pd
import csv
import os,json
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

#os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [ ]:
!pip install biopython
import Bio
from Bio import SeqIO

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
os.listdir("/content/gdrive/My Drive/NPPI")
path="/content/gdrive/My Drive/NPPI"

In [ ]:
def JSLD(path,fn):
  with open(path+fn,"r") as f:
    D=json.load(f)
  return D

In [ ]:
SD={}
with open(path+"/FastaSeq/Protein.fasta") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        pid=(record.id).split("|")[1]
        seqn=str(record.seq)
        SD[pid]=seqn

print(len(SD))
      

2883


In [ ]:
def SeqEncoding(P2SQ):
    """  Encode Map   """
    FasD={"A":"#","G":"#","V":"#","C":"$","D":"&","E":"&","F":"@","I":"@","L":"@","P":"@",\
          "H":"*","N":"*","Q":"*","W":"*","K":"+","R":"+","M":"?","S":"?","T":"?","Y":"?"}
    NC=["B","J","O","U","X","Z"]## Removing 
    P2ESQ={}
    for pid in P2SQ:
        seq=P2SQ[pid]
        for xi in NC:
            if xi in seq:
                seq=seq.replace(xi,"")
        EnSeq=[str(FasD[x]) for x in seq]
        P2ESQ[pid]="".join(EnSeq)
    return P2ESQ

In [ ]:
TrS=SeqEncoding(SD)
print(len(TrS))

2883


In [ ]:
#P30504

In [ ]:
TrS['P30504']

'?+#?#@+#@@@@@?##@#@?&?*#$?*??+?@&?##?+@#+#&@+@@?##?#&&?*@#+@&?&##?@+#&@+#@*#&*&#@&?*&+&?*+?++*#*#&+#?@+*@+#??**?&&#?*?@*+??#$&@#@&#+@@+#?&*?#?&#+&?@#@*&&@+?*?##&?##*@?*++@&##+##&*@+#?@&#?$#&*@++?@&*#+&?@*+#&@@+?*#?**@@?&*&#?@+$*#@#@?@#&@?@?**+&#&&*?*&?&@#&?+@##&#?@*+*#####@?#*&*+??$*?**&#@*&@@?@?*&@??*@?@@@?#@###@##@##@##@####?#??$+++??##+##?$?*##$?*?#*#?&&?@@?$+#'

In [ ]:
global TrAA
def genRedSeqFeat():    
    A=["#","$","@","&","*","+","?"]
    TrAA=[]
    for i in A:
        for j in A:
            for k in A:
                TrAA.append(i+j+k)
    return TrAA
TrAA= genRedSeqFeat()  

def getFrq(seq):
    global TrAA; FrqMA=[]
    for fet in TrAA:
        FrqMA.append(seq.count(fet))
    return(FrqMA)
    
#cmpMinLen = lambda A, B :len([i for i in range(len(A)) if min(A[i],B[i])>0])

def MutualFreq(A,B):
    L=[]
    for i in range(len(A)):
        x=A[i];y=B[i];sc=0
        if (x and y):sc=-(min(x,y)/max(x,y))* np.log(min(x,y)/(x+y))
        L.append(sc)
    return L


In [ ]:
TrSD={}
for k in TrS:
  TrSD[k]=getFrq(TrS[k])
print(len(TrSD))
print(len(TrSD[k]))

2883
343


In [ ]:
with open(path+"/traindata/ProteinData/PData.json","w") as f:
  json.dump(TrSD,f)

In [ ]:
def dataGen(L):
  global TrSD
  Data=[];Label=[]
  
  for x in L:
    p1=x[0];p2=x[1];cl=int(x[2])
    d1= MutualFreq(TrSD[p1],TrSD[p2])
    Data.append(d1)
    Label.append(cl)
  Dx={'data':Data,'label':Label}
  return [Data,Label]
 

In [ ]:
def writeFile(fpath,fname, Dt):
  with open("{}{}.json".format(fpath,fname),"w") as f:
    json.dump(Dt,f)
  print("File write Complete")

In [ ]:
fnA="/traindata/ITR-1/SetA.json"
fnB="/traindata/ITR-1/SetB.json"
A=JSLD(path,fnA)
B=JSLD(path,fnB)
print(len(A), len(B))


26840 26840


In [ ]:
3355*5

16775

In [ ]:
"""
Ax=dataGen(A)  #  /traindata/ITR-1/SetA.json
writeFile(path+'/traindata/ITR-1/',"SetAD",Ax)
Bx=dataGen(B)
writeFile(path+'/traindata/ITR-1/',"SetBD",Bx)
"""

'\nAx=dataGen(A)  #  /traindata/ITR-1/SetA.json\nwriteFile(path+\'/traindata/ITR-1/\',"SetAD",Ax)\nBx=dataGen(B)\nwriteFile(path+\'/traindata/ITR-1/\',"SetBD",Bx)\n'

In [ ]:
from datetime import datetime
from joblib import dump, load
import os
from pathlib import Path
from sklearn import svm
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, roc_curve, auc, average_precision_score, recall_score, log_loss,\
accuracy_score,roc_auc_score,precision_recall_curve,f1_score,matthews_corrcoef

In [ ]:
classifiersX = {
  'SVM' : svm.SVC(C=2.0, kernel='rbf',gamma=0.0005, probability=True),#svm.SVC(kernel='linear', C=0.01),
  'RF' : RandomForestClassifier(n_estimators=192,min_samples_leaf = 75),  # random_forest
  #'decision_tree' : DecisionTreeClassifier(random_state=0),
  'KNN' : KNeighborsClassifier(n_neighbors=7),  # kneighbors
}

In [ ]:
def getFalsePos(predX,actualY,Ix):
  FPI=[]
  TP=FP=TN=FN=0
  for i,a in enumerate(actualY):
    if a==0:
      #print(i,a, predX[i])
    #if a==predX[i] and a==0:  it was  logically wrong for false positive(FP) computation. FP is those which are actually negative but predicted as positive
      
      if (predX[i]==1): # the corrected one
        FPI.append(i);FP+=1
      else:TN+=1

    elif (a==1):
      if (predX[i]==a): TP+=1
      else: FN+=1

  print("Total False Positive:fold-{}:: TP:{},FP:{},TN:{},FN:{}".format(Ix,TP,FP,TN,FN))
  return TP,FP,TN,FN


In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
D=[[1,2],[3,4],[5,6],[7,8,9]]
XD= list(chunks(D, 2))
print(XD)


[[[1, 2], [3, 4]], [[5, 6], [7, 8, 9]]]


In [ ]:
def getPartition(D, L):
  p=sum(L); n=len(L)-p
  r=int(n/p)
  print (r, p, n)
  DL=[]; PS=[]; NS=[]; CL=[]
  for i,a in enumerate(L):
    if a==1: PS.append(D[i])
    else: NS.append(D[i])
  #print(len(NS))
  NL=list(chunks(NS,p))
  #print(len(NL))
  

  for k in NL:
    #print(len(k))
    
    XL=[]; YL=[]
    XL=PS+k
    #XL+=k
    YL=[1 for x in range (len(PS))]+[0 for x in range (len(k))]
    #YL+=[0 for x in range (len(k))]
    DL.append(np.array(XL)); CL.append(np.array(YL))
    #print (len(XL), len(YL), len(k), len(PS))
    #XL=[];YL=[]
    
  #print(len(DL), len(CL))
  return DL, CL

In [ ]:
def writeTFile1(fpath,S):
  f=open("{}".format(fpath),"a")
  f.write(S);f.close()
  

In [ ]:
def RunClassifier(Xi,Yi,Set, itr):
  dI=np.array(Xi[0])
  lI=np.array(Xi[1])
  testD=np.array(Yi[0]);testL=np.array(Yi[1])
  print(dI.shape,": " , lI.shape)
  print(testD.shape,": " , testL.shape)
  DL, CL = getPartition(Xi[0], Xi[1])
  PRd={}
  S=""
  for i,dataI in enumerate(DL): 
    labelI=CL[i]
    clf=classifiersX['RF']
    clf.fit(dataI, labelI)
    dump(clf, path+"/traindata/ITR-{}/{}-Model.joblib".format(itr,Set))
    

    prediction = clf.predict(testD)
    TP,FP,TN,FN=getFalsePos(prediction,testL, i)
    PRd[i]=[[int(x) for x in (prediction)],Yi[1]]
    # PRd first index == fold

    probas_RF = clf.predict_proba(testD)

    AUC=roc_auc_score(testL,probas_RF[:, 1])
    ACCU=accuracy_score(testL,prediction)
    preRF, recRF, thrRF = precision_recall_curve(testL, probas_RF[:,1])
    AUPRC = auc(recRF, preRF)
    f1_s=f1_score(testL, prediction)
    mcc_s=matthews_corrcoef(testL, prediction)
    DScore={"AUC":AUC, "ACCU": ACCU, "AUPRC": AUPRC, "F1": f1_s, "MCC": mcc_s }

    S+="{},{},{},{},{},{},{},{},{},{},{},{}\n".format(itr,Set,i+1, AUC, ACCU,AUPRC,f1_s,mcc_s,TP,FP,TN,FN)
    
    #S="{}:AUC:{},ACCU:{},auPRC:{},F1:{},MCC;{}\n".format(i, AUC, ACCU,AUPRC,f1_s,mcc_s)
    # dump DScore as json
    #print(S)
    #########
    #rS="{},{},{}".format(itr,Set,S)
    
    #writeTFile1(path+'/traindata/allResult.csv',rS)
  
    #########


  with open(path+"/traindata/ITR-{}/{}-predC.json".format(itr,Set),"w") as f:
    json.dump(PRd,f)
  return FP,S

In [ ]:
def writeTFile(fpath,fname,S):
  f=open("{}{}.txt".format(fpath,fname),"w")
  f.write(S);f.close()

In [ ]:
def UpdateItr(NS,B):
  UB=[]
  for i, a in enumerate(NS):
      UB.append(B[a])
  print("NewData Size= ", len(UB))
  return UB


In [ ]:
x=[1,1,1]
xyz=x[0]
for k in range(len(x)-1):
  xyz = xyz and x[k+1]
print(xyz)

1


In [ ]:
def loadFP(Set, itr):
  #Set="B"
  NewSet=[]

  count=0; iz=0
  with open(path+"/traindata/ITR-{}/{}-predC.json".format(itr,Set),"r") as f:
    PRd= json.load(f)
  AC=PRd['0'][1]
  for k,a in enumerate(PRd['0'][0]):
    
    if (AC[k]==0):
      nz=PRd['0'][0][k]
      for ij in range(len(PRd)-1):
        nz = nz and PRd[str(ij+1)][0][k]
      #nz= PRd['0'][0][k] and PRd["1"][0][k] and PRd["2"][0][k] and PRd["3"][0][k] and PRd["4"][0][k] #number of false positives. all 1
      
      if nz==1:
        count+=1
      else:
        NewSet.append(k)
        iz+=1

    else:
      NewSet.append(k)

  print ("False Pos: {} ={}".format(Set,count))
  return NewSet
  
#updated negative set has become 16246
#pos still 3355 
#removed 529 values false pos
#NS=loadFP("B")

In [ ]:
def loadFPnew(Set, itr):
  #Set="B"
  NewSet=[]

  count=0; iz=0
  with open(path+"/traindata/ITR-{}/{}-predC.json".format(itr,Set),"r") as f:
    PRd= json.load(f)
  AC=PRd['0'][1]
  for k,a in enumerate(PRd['0'][0]):
    
    if (AC[k]==0):
      nz=PRd['0'][0][k]
      nzi=nz
      for ij in range(len(PRd)-1):
        nz = nz and PRd[str(ij+1)][0][k]
        nzi+=PRd[str(ij+1)][0][k]
      #nz= PRd['0'][0][k] and PRd["1"][0][k] and PRd["2"][0][k] and PRd["3"][0][k] and PRd["4"][0][k] #number of false positives. all 1
      
      if nzi>2:
        count+=1
      else:
        NewSet.append(k)
        iz+=1

    else:
      NewSet.append(k)

  print ("False Pos: {} ={}".format(Set,count))
  return NewSet
  
#updated negative set has become 16246
#pos still 3355 
#removed 529 values false pos
#NS=loadFP("B")

In [ ]:
def Process(A,B,itr):
  Ax=dataGen(A)  #  /traindata/ITR-1/SetA.json
  writeFile(path+'/traindata/ITR-{}/'.format(itr),"SetAD",Ax)
  Bx=dataGen(B)
  writeFile(path+'/traindata/ITR-{}/'.format(itr),"SetBD",Bx)
  FPb,SB=RunClassifier(Ax,Bx,"B",itr)
  FPa,SA=RunClassifier(Bx,Ax,"A",itr)

  #writeTFile(path+'/traindata/ITR-{}/'.format(itr),"ResSetB",SB)

  #writeTFile(path+'/traindata/ITR-{}/'.format(itr),"ResSetA",SA)
  
  #rSB="{},{},{}".format(itr,"B",SB)
  #rSA="{},{},{}".format(itr,"A",SA)

  writeTFile1(path+'/traindata/allResult.csv',SB)
  
  writeTFile1(path+'/traindata/allResult.csv',SA)
 
  print("B")
  NS=loadFP("B", itr)
  nB=UpdateItr(NS,B)

  writeFile(path+'/traindata/ITR-{}/'.format(itr+1),"SetB",nB)
  print("A")
  NS=loadFP("A", itr)
  nA=UpdateItr(NS,A)

  writeFile(path+'/traindata/ITR-{}/'.format(itr+1),"SetA",nA)
  return nA, nB

In [ ]:
fnA="/traindata/ITR-21/SetA.json"
fnB="/traindata/ITR-21/SetB.json"
A=JSLD(path,fnA)
B=JSLD(path,fnB)
print(len(A), len(B))

20485 20445


In [ ]:
#S="itr,Set,Fold, AUC, ACCU,AUPRC,f1_s,mcc_s,TP,FP,TN,FN \n"
#writeTFile(path+'/traindata/allResult.csv',rSA)
#f=open(path+'/traindata/allResult.csv',"w")
#f.write(S);f.close()

for itr in range(22,51):
  resDir=path+"/traindata/ITR-{}".format(itr)
  resDir1=path+"/traindata/ITR-{}".format(itr+1)
  if not os.path.exists(resDir):
    os.makedirs(resDir)
  if not os.path.exists(resDir1):
    os.makedirs(resDir1)
  print ("A= {}, B={}".format(len(A),len(B)))
  A,B=Process(A,B,itr)
  print("----------Iteration : {}-------------".format(itr))

A= 20485, B=20445
File write Complete
File write Complete
(20485, 343) :  (20485,)
(20445, 343) :  (20445,)
5 3355 17130
Total False Positive:fold-0:: TP:2244,FP:615,TN:16475,FN:1111
Total False Positive:fold-1:: TP:2261,FP:861,TN:16229,FN:1094
Total False Positive:fold-2:: TP:2206,FP:788,TN:16302,FN:1149
Total False Positive:fold-3:: TP:2255,FP:903,TN:16187,FN:1100
Total False Positive:fold-4:: TP:2235,FP:622,TN:16468,FN:1120
Total False Positive:fold-5:: TP:3355,FP:17090,TN:0,FN:0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


(20445, 343) :  (20445,)
(20485, 343) :  (20485,)
5 3355 17090
Total False Positive:fold-0:: TP:2232,FP:714,TN:16416,FN:1123
Total False Positive:fold-1:: TP:2226,FP:552,TN:16578,FN:1129
Total False Positive:fold-2:: TP:2221,FP:939,TN:16191,FN:1134
Total False Positive:fold-3:: TP:2281,FP:737,TN:16393,FN:1074
Total False Positive:fold-4:: TP:2260,FP:734,TN:16396,FN:1095
Total False Positive:fold-5:: TP:3355,FP:17130,TN:0,FN:0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


B
False Pos: B =130
NewData Size=  20315
File write Complete
A
False Pos: A =105
NewData Size=  20380
File write Complete
----------Iteration : 22-------------
A= 20380, B=20315
File write Complete
File write Complete
(20380, 343) :  (20380,)
(20315, 343) :  (20315,)
5 3355 17025
Total False Positive:fold-0:: TP:2223,FP:618,TN:16342,FN:1132
Total False Positive:fold-1:: TP:2294,FP:928,TN:16032,FN:1061
Total False Positive:fold-2:: TP:2212,FP:788,TN:16172,FN:1143
Total False Positive:fold-3:: TP:2225,FP:763,TN:16197,FN:1130
Total False Positive:fold-4:: TP:2241,FP:580,TN:16380,FN:1114
Total False Positive:fold-5:: TP:3355,FP:16960,TN:0,FN:0


...

(20209, 343) :  (20209,)
(20250, 343) :  (20250,)
5 3355 16854
Total False Positive:fold-0:: TP:2244,FP:640,TN:16255,FN:1111
Total False Positive:fold-1:: TP:2256,FP:646,TN:16249,FN:1099
Total False Positive:fold-2:: TP:2209,FP:950,TN:15945,FN:1146
Total False Positive:fold-3:: TP:2253,FP:566,TN:16329,FN:1102
Total False Positive:fold-4:: TP:2267,FP:702,TN:16193,FN:1088
Total False Positive:fold-5:: TP:3355,FP:16895,TN:0,FN:0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


B
False Pos: B =156
NewData Size=  20053
File write Complete
A
False Pos: A =87
NewData Size=  20163
File write Complete
----------Iteration : 24-------------
A= 20163, B=20053
File write Complete
File write Complete
(20163, 343) :  (20163,)
(20053, 343) :  (20053,)
5 3355 16808
Total False Positive:fold-0:: TP:2217,FP:495,TN:16203,FN:1138
Total False Positive:fold-1:: TP:2233,FP:801,TN:15897,FN:1122
Total False Positive:fold-2:: TP:2205,FP:697,TN:16001,FN:1150
Total False Positive:fold-3:: TP:2237,FP:729,TN:15969,FN:1118
Total False Positive:fold-4:: TP:2211,FP:578,TN:16120,FN:1144
Total False Positive:fold-5:: TP:3355,FP:16698,TN:0,FN:0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


(20053, 343) :  (20053,)
(20163, 343) :  (20163,)
4 3355 16698
Total False Positive:fold-0:: TP:2225,FP:588,TN:16220,FN:1130
Total False Positive:fold-1:: TP:2268,FP:654,TN:16154,FN:1087
Total False Positive:fold-2:: TP:2191,FP:961,TN:15847,FN:1164
Total False Positive:fold-3:: TP:2254,FP:637,TN:16171,FN:1101
Total False Positive:fold-4:: TP:2287,FP:950,TN:15858,FN:1068
B
False Pos: B =87
NewData Size=  19966
File write Complete
A
False Pos: A =138
NewData Size=  20025
File write Complete
----------Iteration : 25-------------
A= 20025, B=19966
File write Complete
File write Complete
(20025, 343) :  (20025,)
(19966, 343) :  (19966,)
4 3355 16670
Total False Positive:fold-0:: TP:2246,FP:531,TN:16080,FN:1109
Total False Positive:fold-1:: TP:2252,FP:796,TN:15815,FN:1103
Total False Positive:fold-2:: TP:2215,FP:767,TN:15844,FN:1140
Total False Positive:fold-3:: TP:2230,FP:647,TN:15964,FN:1125
Total False Positive:fold-4:: TP:2332,FP:699,TN:15912,FN:1023
(19966, 343) :  (19966,)
(20025, 343)

In [ ]:
####NOT REQ

In [ ]:
print("B")
NS=loadFP("B")
nB=UpdateItr(NS,B)

writeFile(path+'/traindata/ITR-2/',"SetB",nB)
print("A")
NS=loadFP("A")
nA=UpdateItr(NS,A)

writeFile(path+'/traindata/ITR-2/',"SetA",nA)

In [ ]:
#Second Iteration
A=nA
B=nB


In [ ]:
#iz+count
3355*6 - 529
19601-3355


In [ ]:
print(1 and 1 and 0)

In [ ]:
############################
"""
FPa,SA=RunClassifier(Bx,Ax,"A")
nA=UpdateItr(FPa,A)

writeTFile(path+'/traindata/ITR-1/',"ResSetA",SA)
writeFile(path+'/traindata/ITR-2/',"SetA",nA)
"""